<a href="https://colab.research.google.com/github/yasstake/rusty-bot/blob/main/tutorial/channel_breakout/breakout_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BackTestのチュートリアル

<a target="_blank" href="https://colab.research.google.com/github/yasstake/rusty-bot/blob/main/tutorial/breakout_agent.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


## 1. 実装するBotのロジック

Ukiさんの「オープニングレンジ・ブレイクアウト」をお借りして、バックテストのtutorialを作成しました（公開くさだって感謝）。

Binanceのspot/BTCBUSD用に書いていますが、PublicAPIのみ利用のためアカウントは不要で試すことができます。



<blockquote class="twitter-tweet"><p lang="ja" dir="ltr">お待ちかねのロジックを公開。<br>(元祖)ドテン君はブレイクアウトだが通常のHLチャネルブレイクではない。その手法は「オープニングレンジ・ブレイクアウト」と呼ばれるものである。文章での説明は面倒なのでhohetoとの会議資料をそのまま添付する。ストラテジーに関する質問には応対できません。 <a href="https://t.co/LB6mdxVZZo">pic.twitter.com/LB6mdxVZZo</a></p>&mdash; UKI (@blog_uki) <a href="https://twitter.com/blog_uki/status/981768546429448192?ref_src=twsrc%5Etfw">April 5, 2018</a></blockquote> <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>

![](https://pbs.twimg.com/media/DZ_xJQFUMAEOrpS?format=jpg&name=900x900)

## 2. BreakOutAgentの実装

日本語で読み解くと以下のようになります。

「シグナル点灯した次の足の始値」ではなく、シグナル点灯直後の最良値でエントリーすることに変更した以外はほぼ忠実に書き下しています。

1. `Agent.on_clock`を10分毎に呼び出す。
2.

3. `Agent.on_clock`内で以下の処理を行う。

    1. 前処理
       
       前回のon_clock中でのオーダーが処理中の場合はなにもしない（リターン）
    2. Long/Short判定
      * 現在時刻から2時間足を6本取得する。`session.ohlcv`（6本目の最後の足は未確定足。10分毎に呼ばれるたびにupdateされる）
      (ohlcv[0]-ohlcv[5]へ値を格納)
      * 0-4本目の足のレンジ幅（高値ー安値）の平均値を計算`(ohlcv['high]-ohlcv['low']).mean() * K`し、Kをかけたものを'range_width'へ保存する。
      * 最新足（未確定）の始値〜高値を計算する('diff_high')
      * 最新足（未確定）の始値〜安値を計算する('diff_low')
      * 'range_width', 'diff_high', 'diff_low'の結果からLong/Short判定
    3. オーダー執行
       1. Long判定のとき
          * すでにLongポジションがある場合は何もしない。
          * Shortポジションがあった場合はドテン（最良値、倍サイズ）
          * 上記に当てはまらない場合は最良値、通常サイズでLong(Buy)
       2. Short判定のとき
           * すでにShortポジションがある場合は何もしない
           * Shortポジションがあった場合はドテン（最良値、倍サイズ）
           * 上記に当てはまらない場合は最良値、通常サイズでShort(Sell)

## 2. 依存ライブラリのインストール

In [1]:
# rbot拡張ライブラリのインストール（２回目は不要です）
! pip install --upgrade pip
! pip install --upgrade rbot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 12.1 MB/s eta 0:00:00


In [2]:
# 関連ライブラリのインストール
! pip install pyarrow
! pip install --upgrade polars
! pip install plotly
! pip install nbformat
! pip install numpy
! pip install pandas
! pip install json2html

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.3/26.3 MB 46.8 MB/s eta 0:00:00
  Attempting uninstall: polars
    Found existing installation: polars 0.20.2
    Uninstalling polars-0.20.2:
      Successfully uninstalled polars-0.20.2
  Preparing metadata (setup.py) ... done
  Created wheel for json2html: filename=json2html-1.3.0-py3-none-any.whl size=7593 sha256=9eb078371caf657533e6cdc041f70ba0664cf0ec64663e9f5286fb887e6ef44d
  Stored in directory: /root/.cache/pip/wheels/e0/d8/b3/6f83a04ab0ec00e691de794d108286bb0f8bcdf4ade19afb57
Successfully built json2html


In [3]:
# rbotは今回提供するbacktestライブラリ
import rbot
from rbot import Runner

from rbot import time_string
from rbot import DAYS_BEFORE

## 3. break out Agent(bot)の実装

#### 時間の扱い

時刻について、以下の基準で設計しています。
* 時刻（ある一点を指す）: UTCでエポックタイムからのマイクロ秒(μs)。時刻０はfromの時は過去全部、toのときは将来全部（≒現在時刻）
* 期間：ほとんどで秒(s)。ログのダウンロード`Market.download`だけ日(day)
  * 例：ohlcvの足の幅などは秒を利用。

#### Botの実装すべき必須メソッド

３つのメソッドを実装するとフレームワークが適宜呼び出してくれます。
* `on_clock(self, session, current_time_us)` 定期的に呼び出される関数です。`time_us`は現在時刻です。`session`クラスを使ってローソク足`ohlcv`をとったりオーダーの発行ができます。
* `on_tick(self, session, side, price, size)` 全ログイベントを受け取ります。
* `on_update(self, session, updated_order)`オーダーが約定したり失効した場合に呼ばれます。'updated_order'の中に更新後のオーダ情報が含まれています。



In [4]:
class BreakOutAgent:
    """
        Agentのクラス名は任意。
        on_tick, on_clock, on_updateを実装するとフレームワークから呼び出される。
    """

    def __init__(self, param_K=1.6):
            self.param_K = param_K  # パラメターKを設定する。
            self.offset = 10.0       # 指値のオフセット

    def on_init(self, session):
        print("execute mode = ", session.execute_mode)
        session.clock_interval_sec = 60*60*2  # 2時間ごとにon_clockを呼び出すように設定する。

    def on_clock(self, session, time_us):
        """ Botのメインロジック。設定した秒数毎に呼ばれる """
        # 古いオーダーをキャンセルする。

        # 前処理/ 前回のon_clock中でのオーダーが処理中の場合はなにもしない（リターン）
        if session.buy_orders or session.sell_orders:
            session.expire_order(60*60*2)  # 2時間以上経過したオーダーをキャンセルする。

        ############   メインロジック  ######################
        ohlcv_df = session.ohlcv(60*60*2, 6)  # 2時間足(60*60*2sec)を６本取得。 最新は６番目。ただし未確定足
        if len(ohlcv_df) < 6:           # データが過去６本分そろっていない場合はなにもせずリターン
            return

        ohlcv5 = ohlcv_df[:-2]       # 過去５本足（確定）
        range_width = (ohlcv5['high'] - ohlcv5['low']).mean() * self.param_K  # 価格変動レンジの平均を計算 * K

        # Long/Short判定
        ohlcv_latest = ohlcv_df[-2:-1]     # 最新足１本(未確定)
        diff_low   =   (ohlcv_latest['open'][0] - ohlcv_latest['low'][0])
        detect_short  = range_width < diff_low

        diff_high  = - (ohlcv_latest['open'][0] - ohlcv_latest['high'][0])
        detect_long = range_width  < diff_high

        ##########  メインロジック中に利用したindicatorのロギング（あとでグラフ化するため保存）    ##############
        session.log_indicator('diff_low', diff_low)
        session.log_indicator('diff_high', diff_high)
        session.log_indicator('range_width', range_width)

        ##########　執行戦略（順方向のポジションがあったら保留。逆方向のポジションがのこっていたらドテン）#########
        ORDER_SIZE = 0.01     # 標準オーダーサイズ(ドテンの場合はx2)

        session.log_indicator('position', session.position)

        if detect_long and (session.position <= 0):
            if session.sell_orders:
                for order in session.sell_orders:
                    session.cancel_order(order.order_id)

            bit_edge, ask_edge = session.last_price
            session.limit_order('Buy', bit_edge - self.offset, ORDER_SIZE + (-session.position))

        if detect_short and (0 <= session.position): # short判定のとき
            if session.buy_orders:
                for order in session.buy_orders:
                    session.cancel_order(order.order_id)

            bit_edge, ask_edge = session.last_price
            session.limit_order('Sell', ask_edge + self.offset, ORDER_SIZE + session.position)

    # 全Tick受け取りたい時は on_tick を実装する。
    #def on_tick(self, time, session, side, price, size):
    #    pass

    #約定イベントを受け取りたい時は on_updateを実装する。
    def on_update(self, session, order_result):
        print(order_result)

    def on_account_update(self, session, account):
        print(account)


## 4. Exchange/Market オブジェクトの作成 & データのロード

### Binanceオブジェクトを生成。

バックテストなので本番ネットを選択肢、`Binance`か`Bybit`のexchangeオブジェクトを生成し、`Config`オブジェクトをあたえてMarketをひらきます。

その後、marketオブジェクトをつかって取引履歴をダウンロードします。


In [5]:
# Binanceマーケットを指定します。(BinanceかBybitのどちらかを選択してください。)
from rbot import Binance
from rbot import BinanceConfig


binance_exchange = Binance(production=True)     # 本番ネットのデータを取得します。

config = BinanceConfig.BTCUSDT          # BTC/USDTの市況情報を取得します。
binance_market = binance_exchange.open_market(config)   # BTCUSDTの市況情報を取得するためのマーケットを開きます。


In [6]:
# Bybitマーケットを指定します。(binanceかbybitのどちらか一方を選択してください)
from rbot import Bybit
from rbot import BybitConfig


bybit_exchange = Bybit(production=True)     # 本番ネットのデータを取得します。

config = BybitConfig.BTCUSDT          # BTC/USDTの市況情報を取得します。
bybit_market = bybit_exchange.open_market(config)   # BTCUSDTの市況情報を取得するためのマーケットを開きます。


In [7]:
#exchange = bybit_exchange
#market = bybit_market
exchange = binance_exchange
market = binance_market

### データのダウンロード

最初にダウンロードメソッドをつかってデータをダウンロードします。

#### ダウンロードメソッド
```
BinanceMarket.download_archive(ndays, force=False, verbose=True)
```

##### パラメータ
* `ndays` 何日前のデータからダウンロードするかを指定します。
* `forde` オプション：TureにするとローカルDBにデータがあっても再ダウンロードします。
* `verbose` オプション:ダウンロード状況を印刷します（ディフォルトは表示）

##### 戻り値
* ダウンロードされたレコード数


In [8]:
#from rbot import init_debug_log;
#init_debug_log()

# 過去２日分のデータをダウンロード
market.download_archive(
    ndays=10,       #ダウンロード日数
    verbose=True    #ダウンロード状況を表示
)

18973970

In [9]:
# かなり巨大なデータベースができあがります。１日分で数百MB。
# DBを削除する場合は以下のコマンドを実行してください
# なお最後に表示されたファイルを別途OSから削除してください。
#binance.drop_table()

# DBのある場所を表示します。かなり大きなファイルになりますので不要になった場合手動で消してください。
#binance.file_name

## 5. バックテスト実行

In [10]:

from rbot import NOW, DAYS
from rbot import Runner

agent = BreakOutAgent()  # エージェントを作成する。パラメターKはデフォルトの1.6を利用する。

runner = Runner()

session = runner.back_test(
                exchange=exchange,  # 取引所を指定する。
                market=market,     # マーケットを指定する。
                agent=agent,    # エージェントを指定する。
                start_time=NOW()-DAYS(10),   # 開始時間を指定する。0は最初から。10日前から実行を指定。
                end_time=0,     # 終了時間を指定する。0は最後まで。
                verbose=True    # ログを表示するかどうか
)


execute mode =  Dummy
{"category":"SPOT","symbol":"BTCUSDT","create_time":1711533600000000,"status":"New","order_id":"BreakOutAgent-Us6DmB0001","client_order_id":"BreakOutAgent-Us6DmB0001","order_side":"Sell","order_type":"Limit","order_price":"70067.6","order_size":"0.01","remain_size":"0.01","transaction_id":"","update_time":1711533600001000,"execute_price":"0.0","execute_size":"0.0","quote_vol":"0.0","commission":"0.0","commission_asset":"USDT","is_maker":true,"message":"","commission_home":"0.0","commission_foreign":"0.0","home_change":"0.0","foreign_change":"0.0","free_home_change":"0.0","free_foreign_change":"-0.01","lock_home_change":"0.0","lock_foreign_change":"0.01","open_position":"0.0","close_position":"0.0","position":"0.0","profit":"0.0","fee":"0","total_profit":"0","log_id":1}
{"category":"SPOT","symbol":"BTCUSDT","create_time":1711533600000000,"status":"Filled","order_id":"BreakOutAgent-Us6DmB0001","client_order_id":"BreakOutAgent-Us6DmB0001","order_side":"Sell","order_t

### 実行結果の分析

`session.log`にLoggerオブジェクトとして結果が保存されています。
詳しくは[Loggerのマニュアル](../manual.ipynb)を参照ください。

In [11]:
# loggerオブジェクトの取得
log = session.log

In [12]:
import polars as pl
pl.Config(fmt_str_lengths=50)

# オーダログの表示
orders = log.orders
orders

log_id,symbol,update_time,create_time,status,order_id,client_order_id,order_side,order_type,order_price,order_size,remain_size,transaction_id,execute_price,execute_size,quote_vol,commission,commission_asset,is_maker,message,commission_home,commission_foreign,home_change,foreign_change,free_home_change,free_foreign_change,lock_home_change,lock_foreign_change,open_position,close_position,position,profit,fee,total_profit,sum_profit
i64,str,datetime[μs],datetime[μs],str,str,str,str,str,f64,f64,f64,str,f64,f64,f64,f64,str,bool,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,"""BTCUSDT""",2024-03-27 10:00:00.001,2024-03-27 10:00:00,"""New""","""BreakOutAgent-Us6DmB0001""","""BreakOutAgent-Us6DmB0001""","""Sell""","""Limit""",70067.6,0.01,0.01,"""""",0.0,0.0,0.0,0.0,"""USDT""",true,"""""",0.0,0.0,0.0,0.0,0.0,-0.01,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"""BTCUSDT""",2024-03-27 10:00:07.187,2024-03-27 10:00:00,"""Filled""","""BreakOutAgent-Us6DmB0001""","""BreakOutAgent-Us6DmB0001""","""Sell""","""Limit""",70067.6,0.01,0.0,"""BreakOutAgent-0001""",70067.6,0.01,700.676,0.0,"""USDT""",true,"""""",0.0,0.0,700.676,-0.01,700.676,-0.01,0.0,-0.01,-0.01,0.0,-0.01,0.0,0.0700676,-0.070068,-0.070068
3,"""BTCUSDT""",2024-03-27 16:00:00,2024-03-27 16:00:00,"""New""","""BreakOutAgent-Us6DmB0002""","""BreakOutAgent-Us6DmB0002""","""Buy""","""Limit""",68864.0,0.02,0.02,"""""",0.0,0.0,0.0,0.0,"""USDT""",true,"""""",0.0,0.0,0.0,0.0,-1377.28,0.0,1377.28,0.0,0.0,0.0,-0.01,0.0,0.0,0.0,-0.070068
4,"""BTCUSDT""",2024-03-27 16:00:04.007,2024-03-27 16:00:00,"""PartiallyFilled""","""BreakOutAgent-Us6DmB0002""","""BreakOutAgent-Us6DmB0002""","""Buy""","""Limit""",68864.0,0.02,0.01957,"""BreakOutAgent-0002""",68864.0,0.00043,29.61152,0.0,"""USDT""",true,"""""",0.0,0.0,-29.61152,0.00043,0.0,0.00043,-29.61152,0.0,0.0,-0.00043,-0.00957,0.517548,0.002961,0.514587,0.444519
5,"""BTCUSDT""",2024-03-27 16:00:04.034,2024-03-27 16:00:00,"""PartiallyFilled""","""BreakOutAgent-Us6DmB0002""","""BreakOutAgent-Us6DmB0002""","""Buy""","""Limit""",68864.0,0.02,0.00666,"""BreakOutAgent-0003""",68864.0,0.01291,889.03424,0.0,"""USDT""",true,"""""",0.0,0.0,-889.03424,0.01291,0.0,0.01291,-889.03424,0.0,0.00334,-0.00957,0.00334,11.518452,0.088903,11.429549,11.874068
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
59,"""BTCUSDT""",2024-04-04 16:01:23.133,2024-04-04 16:00:00,"""PartiallyFilled""","""BreakOutAgent-Us6DmB0010""","""BreakOutAgent-Us6DmB0010""","""Buy""","""Limit""",67637.49,0.02,0.01551,"""BreakOutAgent-0049""",67637.49,0.00192,129.863981,0.0,"""USDT""",true,"""""",0.0,0.0,-129.863981,0.00192,0.0,0.00192,-129.863981,0.0,0.0,-0.00192,-0.00551,-3.908966,0.012986,-3.921953,15.818214
60,"""BTCUSDT""",2024-04-04 16:01:23.310,2024-04-04 16:00:00,"""PartiallyFilled""","""BreakOutAgent-Us6DmB0010""","""BreakOutAgent-Us6DmB0010""","""Buy""","""Limit""",67637.49,0.02,0.01474,"""BreakOutAgent-0050""",67637.49,0.00077,52.080867,0.0,"""USDT""",true,"""""",0.0,0.0,-52.080867,0.00077,0.0,0.00077,-52.080867,0.0,0.0,-0.00077,-0.00474,-1.567658,0.005208,-1.572866,14.245347
61,"""BTCUSDT""",2024-04-04 16:01:23.921,2024-04-04 16:00:00,"""PartiallyFilled""","""BreakOutAgent-Us6DmB0010""","""BreakOutAgent-Us6DmB0010""","""Buy""","""Limit""",67637.49,0.02,0.01466,"""BreakOutAgent-0051""",67637.49,0.00008,5.4109992,0.0,"""USDT""",true,"""""",0.0,0.0,-5.410999,0.00008,0.0,0.00008,-5.410999,0.0,0.0,-0.00008,-0.00466,-0.162874,0.000541,-0.163415,14.081932


## 6.結果分析


In [13]:
# 損益結果
orders['sum_profit'][-1]

4.49538862

In [14]:
# 利益ピーク
orders['sum_profit'].max()

28.52546674

In [15]:
# 利益最小
orders['sum_profit'].min()

-7.97694924

In [16]:
# １回の取引の最大利益
orders['total_profit'].max()

16.74830795942

In [17]:
# １回の取引の最大損失
orders['total_profit'].min()

-6.56337181182

In [18]:
# オーダー状況分析
orders.group_by(['order_side', 'status']).count()

<ipython-input-18-75c86f370cf8>:2: DeprecationWarning: `count` is deprecated. It has been renamed to `len`.
  orders.group_by(['order_side', 'status']).count()


order_side,status,count
str,str,u32
"""Buy""","""New""",5
"""Buy""","""PartiallyFilled""",29
"""Buy""","""Filled""",5
"""Sell""","""New""",5
"""Sell""","""Filled""",5
"""Sell""","""PartiallyFilled""",14


In [19]:
# オーダーごとに集約
group_by_order = orders.group_by(['order_id']).agg(
    pl.col('symbol').first(),
    pl.col('order_side').first(),
    pl.col('status').last(),
    pl.col('order_price').first(),
    pl.col('order_size').first(),
    pl.col('execute_size').sum(),
    pl.col('update_time').last(),
    pl.col('total_profit').sum()
).sort('update_time')


group_by_order

order_id,symbol,order_side,status,order_price,order_size,execute_size,update_time,total_profit
str,str,str,str,f64,f64,f64,datetime[μs],f64
"""BreakOutAgent-Us6DmB0001""","""BTCUSDT""","""Sell""","""Filled""",70067.6,0.01,0.01,2024-03-27 10:00:07.187,-0.070068
"""BreakOutAgent-Us6DmB0002""","""BTCUSDT""","""Buy""","""Filled""",68864.0,0.02,0.02,2024-03-27 16:00:04.034,11.898272
"""BreakOutAgent-Us6DmB0003""","""BTCUSDT""","""Sell""","""Filled""",68690.01,0.02,0.02,2024-03-27 18:00:05.396,-1.87728
"""BreakOutAgent-Us6DmB0004""","""BTCUSDT""","""Buy""","""Filled""",69842.18,0.02,0.02,2024-03-30 08:00:11.338,-11.661384
"""BreakOutAgent-Us6DmB0005""","""BTCUSDT""","""Sell""","""Filled""",69732.58,0.02,0.02,2024-03-30 22:04:59.989,-1.235465
"""BreakOutAgent-Us6DmB0006""","""BTCUSDT""","""Buy""","""Filled""",69906.61,0.02,0.02,2024-03-31 04:02:19.781,-1.880113
"""BreakOutAgent-Us6DmB0007""","""BTCUSDT""","""Sell""","""Filled""",69605.44,0.02,0.02,2024-04-01 09:14:15.476,-3.150911
"""BreakOutAgent-Us6DmB0008""","""BTCUSDT""","""Buy""","""Filled""",65942.01,0.02,0.02,2024-04-03 18:00:04.462,36.502416
"""BreakOutAgent-Us6DmB0009""","""BTCUSDT""","""Sell""","""Filled""",65601.57,0.02,0.02,2024-04-04 06:00:19.666,-3.535603


In [20]:
lost = len(group_by_order.filter(pl.col('total_profit') < 0))
win = len(group_by_order.filter(pl.col('total_profit') > 0))

print(f'勝ち:{win} 負け:{lost} 勝率:{win/(win+lost)}')

勝ち:2 負け:8 勝率:0.2


## 6. グラフで確認

### まずは該当期間のOHLCV（ローソク足）チャートを書く

#### OHLCVデータの作成


In [21]:
ohlcv = market.ohlcv (
    runner.start_timestamp, # runnerの開始時間
    runner.last_timestamp,  # runnerの終了時間
    60*60*2                 # 2時間足
)

In [22]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.1, row_heights=[0.1, 0.1, 0.1, 0.6],
                    subplot_titles=("indicator", "position", "profit", "candlestick"))

# row 1 (indicator)
diff_low = log['diff_low']
fig.add_trace(go.Scatter(x=diff_low['timestamp'], y=diff_low['diff_low'], name="diff_low"), row=1, col=1)

diff_high = log['diff_high']
fig.add_trace(go.Scatter(x=diff_low['timestamp'], y=diff_low['diff_low'], name="diff_high"), row=1, col=1)

range_width = log['range_width']
fig.add_trace(go.Scatter(x=range_width['timestamp'], y=range_width['range_width'], name="range_width"), row=1, col=1)

orders = log.orders

# row 2 (position)
fig.add_trace(go.Scatter(x=orders['update_time'], y=orders['position'], name="position"), row=2, col=1)


# row 3 (profit)
profit = orders['sum_profit']
fig.add_trace(go.Scatter(x=orders['update_time'], y=orders['sum_profit'], name="profit"), row=3, col=1)

# row 4 (candlestick)
fig.add_candlestick(x=ohlcv['timestamp'], open=ohlcv['open'], high=ohlcv['high'], low=ohlcv['low'], close=ohlcv['close'], row=4, col=1)

# row 4 (order)
buy_orders = orders.filter((orders['order_side'] == 'Buy') & (orders['status'] == 'New'))
fig.add_trace(go.Scatter(x=buy_orders['update_time'], y=buy_orders['order_price'], mode='markers', marker=dict(symbol='arrow-up', color='red', size=10), name="buy"), row=4, col=1)

sell_orders = orders.filter((orders['order_side'] == 'Sell') & (orders['status'] == 'New'))
fig.add_trace(go.Scatter(x=sell_orders['update_time'], y=sell_orders['order_price'], mode='markers', marker=dict(symbol='arrow-down', color='blue', size=10), name="sell"), row=4, col=1)

fig.update_layout(height=800, title_text="Backtest Result")


## 7. まとめ

自分で書くのは百行ぐらいで簡単にバックテストができることが確認できました。

使ってみてご意見お聞かせください。
Twitter(@yasstake)でもgithubでissueあげてもOK.

人気になったらDiscord立ち上げてみたいなー。どおでしょ？


## 8. Next Step

いろいろバックテストしてみよう

#### BreakOutAgentのパラメータ変更
以下のパラメータを変えたらどうなるか確認してみよう
1. パラメータK (現在は1.6きめうち)
2. 指値（現在はbestプライス。
   * buyのときに低い価格で指すと利益は上がるがExpire率が上がる。
   * 高い価格で指すとExpire率が下がるが利益が下がる。（この場合、本来はTakerFeeが取られるが現在はMakerFeeで計算）
3. ロット（現在は0.01 BTC)の変更
4. 2時間足から他の足を試してみる。
5. Clockのタイミングを増やす・へらす（現在は10分）
6. バックテスト期間の延長（現在はBACKTEST_PERIODは5日で設定）

さらに独自のIndicatorをつくり、独自ロジックをつくってバックテストしてみよう（OHLCVをベースに判断するロジックならば問題なく作れるはず）